### SVD

In [1]:
import pandas as pd
import os
import numpy as np

DATA_PATH = '../data/'

ratings_main = pd.read_csv(
    os.path.join(DATA_PATH, 'ratings.csv'),
    dtype={
        'user_uid': np.uint32,
        'element_uid': np.uint16,
        'rating': np.uint8,
        'ts': np.float64,
    }
)

In [2]:
ratings_main.head(6)

,user_uid,element_uid,rating,ts
0,571252,1364,10,4.430517e+07
1,63140,3037,10,4.430514e+07
2,443817,4363,8,4.430514e+07
3,359870,1364,10,4.430506e+07
4,359870,3578,9,4.430506e+07
5,557663,1918,10,4.430505e+07


In [3]:
from sklearn.model_selection import StratifiedShuffleSplit
from numpy import linalg
from scipy.sparse import coo_matrix

In [4]:
%%time

# удаляю из списка фильмов фильмы с менее чем тремя оценками на весь датасет
data = ratings_main.groupby('element_uid').agg({'element_uid': ['count']})
ind = set(data[data['element_uid']['count'] > 2].index)
grouped = ratings_main.groupby('element_uid')
ratings_main = grouped.filter(lambda x: set(x['element_uid']).issubset(ind))

ratings_ds = ratings_main.copy()

# отбираю 30_000 строк для обучения 
# с помощью стратифицированной по элементам выборки
sss = StratifiedShuffleSplit(n_splits=1, train_size=30_000, random_state=0)
for _, (train_index, _) in enumerate(sss.split(ratings_main, ratings_main['element_uid'])):
    ratings_main = ratings_main.iloc[train_index]


# удаляю строки с оценкой фильма 0 (используется шкала от 1 до 10, поэтому это мусор)
ratings_main.drop(labels=ratings_main.groupby('rating').get_group(0).index, 
                  inplace=True)

CPU times: user 966 ms, sys: 43 ms, total: 1.01 s
Wall time: 1.01 s


In [5]:
%%time

users = ratings_main['user_uid'].astype('category')
elements = ratings_main['element_uid'].astype('category')
ratings = ratings_main['rating']
    
coo = coo_matrix((ratings.astype(float), # coo --- матрица оценок
                  (users.cat.codes.copy(), elements.cat.codes.copy()))).toarray()
r_average = round(coo[coo > 0.0].mean(), 2) # заполняем нули (неизвестные значения) средней оценкой
coo[coo == 0] = np.NaN
coo = np.nan_to_num(coo, nan=r_average, copy=False)

CPU times: user 728 ms, sys: 608 ms, total: 1.34 s
Wall time: 1.35 s


In [6]:
%%time
U, Sigma, Vt = linalg.svd(coo, full_matrices=False)

CPU times: user 10min 21s, sys: 2min 4s, total: 12min 25s
Wall time: 1min 44s


In [7]:
def compute_k(Sigma):
    target_sum = 0.9 * np.sum(Sigma)
    cur_sum = 0
    k = 0
    
    while cur_sum < target_sum:
        cur_sum += Sigma[k]
        k += 1
    
    return k

In [8]:
%%time
k = compute_k(Sigma)

CPU times: user 1.55 ms, sys: 317 µs, total: 1.87 ms
Wall time: 4.13 ms


In [9]:
round(len(Sigma)/k, 2) 

9.26

В 9 с лишним раз сократили размерность Sigma, сохранив при этом 90% информации!

In [10]:
# сокращаем размерности разложенных матриц

def rank_k(k):
    U_reduced= np.array(U[:,:k])
    Vt_reduced = np.array(Vt[:k,:])
    Sigma_reduced = np.diag(Sigma[:k])
    Sigma_sqrt = np.sqrt(Sigma_reduced)
    
    return U_reduced @ Sigma_sqrt, Sigma_sqrt @ Vt_reduced

In [11]:
%%time
U_reduced, Vt_reduced = rank_k(k)
ratings_matrix = (U_reduced @ Vt_reduced).round(2)

CPU times: user 9.07 s, sys: 7.91 s, total: 17 s
Wall time: 2.86 s


In [12]:
%%time

# датасет спрогнозированных оценок
# строки соответствуют пользователям, столбцы --- фильмам

ratings_df = pd.DataFrame(ratings_matrix, columns=elements.cat.categories, index=users.cat.categories)

CPU times: user 2.03 ms, sys: 63 µs, total: 2.1 ms
Wall time: 5.27 ms


In [13]:
ratings_df.head(6)

,3,6,7,15,18,26,28,29,30,31,...,10166,10168,10169,10170,10171,10173,10178,10180,10184,10185
1,8.21,8.21,8.21,8.21,8.21,8.21,8.21,8.21,8.21,8.21,...,8.21,8.21,8.21,8.21,8.21,8.21,8.21,8.21,8.21,8.21
17,8.21,8.21,8.21,8.21,8.21,8.21,8.21,8.21,8.21,8.21,...,8.21,8.21,8.21,8.21,8.21,8.21,8.21,8.21,8.21,8.21
20,8.21,8.21,8.21,8.21,8.21,8.21,8.21,8.21,8.21,8.21,...,8.21,8.21,8.21,8.21,8.21,8.21,8.21,8.21,8.21,8.21
25,8.21,8.21,8.21,8.21,8.21,8.21,8.21,8.21,8.21,8.21,...,8.21,8.21,8.21,8.21,8.21,8.21,8.21,8.21,8.21,8.21
72,8.21,8.21,8.21,8.21,8.21,8.21,8.21,8.21,8.21,8.21,...,8.21,8.21,8.21,8.21,8.21,8.21,8.21,8.21,8.21,8.21
105,8.21,8.21,8.21,8.21,8.21,8.21,8.21,8.21,8.21,8.21,...,8.21,8.21,8.21,8.21,8.21,8.21,8.21,8.21,8.21,8.21


In [14]:
%%time

# пример составления рекомендаций для пользователя 231944
user = 231944

# убираем из рекомендаций уже оценённые пользователем элементы
rated_elems = set(ratings_ds[ratings_ds['user_uid']==user]['element_uid'])
columns = set(ratings_df.columns) - rated_elems

pairs = [[ratings_df.loc[user, col], col] for col in columns]
pairs.sort(key=lambda p: p[0], reverse=True)

recs = np.array(pairs).T[1][:10]
recs = recs.astype('I')

CPU times: user 247 ms, sys: 35.8 ms, total: 283 ms
Wall time: 399 ms


In [15]:
recs

array([3349, 3861, 7279, 8569, 3903, 6373,  622, 1003, 2746, 4173],
      dtype=uint32)

In [16]:
ratings_df.loc[user, recs]

3349    8.28
3861    8.25
7279    8.24
8569    8.24
3903    8.23
6373    8.23
622     8.22
1003    8.22
2746    8.22
4173    8.22
Name: 231944, dtype: float64

Посчитаем среднюю квадратичную ошибку на тренировочных (из rating_ds) и тестовых (из ratings_main) данных для пользователей, оценивших более 30-ти элементов:

In [17]:
a = ratings_main.groupby(by='user_uid').count()
users = list(a[a['rating']>30].index) # выбираем пользователей, которые поставили больше 30-ти оценок
users

[24124, 83691, 107891, 110138, 278352, 334052, 412991, 453355]

In [19]:
from sklearn.metrics import mean_squared_error as mse

In [20]:
def calc_rmse(user, ratings_ds, ratings_main):
    elem_train = set(ratings_main[ratings_main['user_uid']==user]['element_uid'])
    elem_test = set(ratings_ds[ratings_ds['user_uid']==user]['element_uid'])
    elem_test &= set(ratings_main['element_uid'].unique()) # рассматриваем только попавшие в таблицу с оценками
    elem_test -= elem_train # убираем элементы из тренировочной выборки
    
    elem_train = list(elem_train)
    elem_test = list(elem_test)
    
    user_ds = ratings_ds[ratings_ds['user_uid']==user]
    
    r_train_pred = ratings_df.loc[user, list(elem_train)]
    r_train_true = [user_ds[user_ds['element_uid']==elem]['rating'] for elem in elem_train]
    r_test_pred = ratings_df.loc[user, list(elem_test)]
    r_test_true = [user_ds[user_ds['element_uid']==elem]['rating'] for elem in elem_test]
    
    rmse_train = round(mse(r_train_true, r_train_pred, squared=False), 2)
    rmse_test = round(mse(r_test_true, r_test_pred, squared=False), 2)
    return [rmse_train, rmse_test]

In [21]:
rmse_arr = []

In [22]:
for user in users:
    rmse_arr.append(calc_rmse(user, ratings_ds, ratings_main))

In [25]:
rmse_arr = np.array(rmse_arr).T
rmse_arr

array([[0.25, 0.39, 0.4 , 0.17, 0.62, 0.13, 0.14, 0.92],
       [1.71, 1.41, 0.65, 2.56, 1.25, 1.56, 2.42, 1.05]])

In [26]:
print('Среднее значение RMSE на тренировочной выборке:',np.mean(rmse_arr[0]))
print('Среднее значение RMSE на тестовой выборке:', np.mean(rmse_arr[1]))

Среднее значение RMSE на тренировочной выборке: 0.3775
Среднее значение RMSE на тестовой выборке: 1.57625


Видно, что ошибка на тренировочной выборке оказалась небольшой, а на тестовой составила около 1.5. То есть для наших пользователей прогноз рекомендательного алгоритма в среднем отличался от реальной оценки на 1.5 единицы из 10. 

Аналогичные вычисления для случайных 10-ти пользователей:

In [28]:
import random

In [33]:
a = ratings_main.groupby(by='user_uid').count()
users = list(a[a['rating']>2].index) # выбираем оценивших хотя бы два фильма
users = random.sample(users, 10) # выбираем 10 случайных пользователей
users

[535086,
 398861,
 109987,
 434600,
 549799,
 133267,
 378841,
 166311,
 255046,
 417917]

In [34]:
rmse_arr = []

In [35]:
for user in users:
    rmse_arr.append(calc_rmse(user, ratings_ds, ratings_main))

In [36]:
rmse_arr = np.array(rmse_arr).T
rmse_arr

array([[0.04, 0.64, 1.05, 1.53, 1.04, 1.02, 1.02, 0.46, 0.02, 1.59],
       [1.76, 1.84, 1.68, 1.55, 2.3 , 2.72, 1.82, 1.96, 1.49, 2.19]])

In [37]:
print('Среднее значение RMSE на тренировочной выборке:',np.mean(rmse_arr[0]))
print('Среднее значение RMSE на тестовой выборке:', np.mean(rmse_arr[1]))

Среднее значение RMSE на тренировочной выборке: 0.841
Среднее значение RMSE на тестовой выборке: 1.9310000000000003
